# 16.1. Sentiment Analysis and the Dataset

# 16.2. Sentiment Analysis: Using Recurrent Neural Networks

# 16.3. Sentiment Analysis: Using Convolutional Neural Networks

# 16.4. Natural Language Inference and the Dataset

# 16.5. Natural Language Inference: Using Attention

# 16.6. Fine-Tuning BERT for Sequence-Level and Token-Level Applications

# 16.7. Natural Language Inference: Fine-Tuning BERT